In [ ]:
%matplotlib inline

In [ ]:
import os
import sys
from collections import defaultdict
import json

import xml.etree.ElementTree as ET
from xml.dom import minidom

import numpy as np
import pandas as pd

import openslide
from PIL import Image
from PIL import ImageDraw

# Convert xml input to a class_label_id dict of shapely polygons
## and display for developer sanity check

In [ ]:
"""
from collections import defaultdict
import json

from xml.dom import minidom
"""
def get_labels_dict(class_labels_id_file_name):
    """ labels_dict = get_labels_dict(class_labels_id) """
    labels_dict = defaultdict(dict)
    try:        
        with open(class_labels_id_file_name, 'r') as fh:
            lines = fh.readlines()
    except:
        print('failed opening: ', class_labels_id_file_name)
        lines = ''
        pass
    
    if len(lines) > 0:
        for line in lines:
            line_list = line.strip().split(',')
            if len(line_list) > 1 and line_list[0] != 'Label':
                labels_dict[line_list[0]] = {'ID': line_list[1], 'Priority': line_list[1]}
                
    return labels_dict


def get_xml_list_of_dicts(xml_file_name):
    """ regions_list = get_xml_list_of_dicts(xml_file_name) """
    xml_obj = minidom.parse(xml_file_name)
    regions_dom = xml_obj.getElementsByTagName("Region")

    regions_list = []
    for reg_dom in regions_dom:
        tmp_dict = {}
        tmp_dict['vertices'] = reg_dom.getElementsByTagName("Vertex")
        tmp_dict['region_Id'] = reg_dom.getAttribute('Id')
        tmp_dict['class_label_text'] = reg_dom.getAttribute('Text')
        tmp_dict['class_label_Id'] = reg_dom.getAttribute('Type')
        tmp_dict['region_geo_shape'] = reg_dom.getAttribute('GeoShape')
        tmp_dict['coords'] = np.zeros((len(tmp_dict['vertices']), 2))
        for i, vertex in enumerate(tmp_dict['vertices']):
            tmp_dict['coords'][i][0] = vertex.attributes['X'].value
            tmp_dict['coords'][i][1] = vertex.attributes['Y'].value
            
        regions_list.append(tmp_dict)
        
    return regions_list


def get_region_Id_dict(xml_file_name, class_labels_id_file_name):
    """ region_id_dict = get_region_Id_dict(xml_file_name, class_labels_id_file_name) """
    # read the xml file into a list of dicts
    regions_list = get_xml_list_of_dicts(xml_file_name)
    # read the csv file into a dict of dicts Label: {ID: x, Priority: y}
    label_dict = get_labels_dict(class_labels_id_file_name)
    # initialize the output dictionary
    region_id_dict = defaultdict(dict)
    
    # enter each region into the output dict
    for region_dict in regions_list:
        # extract and cast the region Priority from the csv file
        region_priority = int(label_dict[region_dict['class_label_text']]['Priority'])
        # construct the rest of the region dict from the xml file
        ridic = {'class_label_text': region_dict['class_label_text'], 
                 'class_label_Id': region_dict['class_label_Id'], 
                 'Priority': region_priority, 
                 'region_geo_shape': region_dict['region_geo_shape'], 
                 'coords': region_dict['coords']}
        
        region_id_dict[int(region_dict['region_Id'])] = ridic
        
    return region_id_dict

def get_region_mask(region_coords, thumbnail_divisor, image_dimensions):
    """ mask_im, img = get_region_mask(region_coords, thumbnail_divisor, image_dimensions) """
    xy_list = (region_coords / thumbnail_divisor).astype(np.int).tolist()
    xy_list = [(p[0], p[1]) for p in xy_list ]
    
    thumbnail_size = tuple((np.array(image_dimensions) / thumbnail_divisor).astype(np.int))
    num_thumb_size = (thumbnail_size[1], thumbnail_size[0])
    
    img = Image.fromarray(np.zeros(num_thumb_size).astype(np.uint8))
    
    drw = ImageDraw.Draw(img)
    drw.polygon(xy_list, fill="white")
    
    mask_im = np.array(img) > 0
    
    return mask_im, img

In [ ]:
    
# Test region Id dict function
zip_tank = '../../DigiPath_MLTK_data/zipTank/wsi_annotation_sample/'
xml_name = os.path.join(zip_tank, 'e39a8d60a56844d695e9579bce8f0335.xml')
c_lab_id_fn = os.path.join(zip_tank, 'class_label_id.csv')

region_id_dict = get_region_Id_dict(xml_name, c_lab_id_fn)
for k_reg_id, v_val in region_id_dict.items():
    print('\n\tregion_Id', k_reg_id)
    for k, v in v_val.items():
        if k == 'coords':
            print('%20s: size = (%i, %i)'%(k, v.shape[0], v.shape[1]), type(v), type(v[0,0]))
        elif k == 'Priority':
            print('%20s: %i'%(k, v))
        else:
            print('%20s: %s'%(k, v))

In [ ]:
# Test Labels Dict function
# zip_tank = '../../DigiPath_MLTK_data/zipTank/wsi_annotation_sample/'
c_lab_id_fn = os.path.join(zip_tank, 'class_label_id.csv')

c_lab_id_fn = os.path.join(zip_tank, 'class_label_id.csv')
lbl_dict = get_labels_dict(c_lab_id_fn)
for k, v in lbl_dict.items():
    print('%20s'%(k), v)
    

In [ ]:
"""
    Test leads to the assumption that the region_Id is the unique identifier for the annotation xml
"""
# Test get_xml_list_of_dicts function -- view input file fields

# xml_name = os.path.join(zip_tank, '54742d6c5d704efa8f0814456453573a.xml')
xml_name = os.path.join(zip_tank, 'e39a8d60a56844d695e9579bce8f0335.xml')

lstod = get_xml_list_of_dicts(xml_name)
keys_list = list(lstod[0].keys())
count = 0
print('\t'.join(keys_list), '\n')
for l in lstod:
    # print(type(l), len(l))
    for k in keys_list:
        if not k in l:
            print('Crash Crash %i not in %i'%(k, count))
        elif k == 'coords':
            print(l['region_Id'], l['class_label_text'], l['class_label_Id'], '\t\t', len(l[k]), '\t', type(l[k]))
        elif k == 'region_geo_shape':
            print(l['region_geo_shape'])
    count += 1
    print()

In [ ]:
im_dir = '../../DigiPath_MLTK_data/RegistrationDevData'
im_file = '54742d6c5d704efa8f0814456453573a.tiff'
image_file_name = os.path.join(im_dir, im_file)
os_obj = openslide.OpenSlide(image_file_name)
image_dimensions = os_obj.dimensions
os_obj.close()

os.listdir(im_dir)

In [ ]:
region_coords = region_id_dict[8]['coords']
thumbnail_divisor = 200
print('image_dimensions', image_dimensions)
mask_im, img = get_region_mask(region_coords, thumbnail_divisor, image_dimensions)

print(type(img))
display(img)